From this repo https://github.com/Aaditya-Singh/E2E-ECPE/tree/main which talks about this paper https://aaditya-singh.github.io/data/ECPE.pdf An End-to-End Network for Emotion-Cause Pair Extraction
Aaditya Singh∗† Shreeshail Hingane† Saim Wani† Ashutosh Modi


This github explains the dataset : https://github.com/naveed92/ecpe_datasets


THis colab is implementation of https://github.com/NVJKKartik/Emo_Cause_Pair/blob/main/E2E_PextE.py


In [ ]:
!pip install gensim


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
base_dir = '/content/drive/MyDrive/E2E-ECPE-main'
import os

# Define the directory path
save_dir = "./save"

# Check if the directory exists, and if not, create it
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

    # Define the directory path
dir = "./data"

# Check if the directory exists, and if not, create it
if not os.path.exists(dir):
    os.makedirs(dir)

In [ ]:
drivePath = '/content/drive/My Drive/NLPproj/ECPESemeval'
traindatasetPath = drivePath + "/Subtask_1_train.json"
testdatasetPath = drivePath + "/Subtask_1_test.json"

In [ ]:
import torch
import json
# Check if GPU is available and select appropriate device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Using device: {device}')


Using device: cuda


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/My Drive/NLPproj/ECPESemeval/Subtask_1_train.json'

utils> funcs.py

In [ ]:
############################################ IMPORT ##########################################################
import sys, os
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score
import torch
from torch import nn
from torch import optim
from torch.nn import functional as F
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

############################################ EMBEDDING LOOKUP ################################################
def embedding_lookup(word_embedding, x):
    '''
    input(s) shape: [num_words, embedding_dim], [batch_size, doc_len, sen_len]
    output shape: [batch_size, doc_len, sen_len, embedding_dim]
    '''
    # x = F.embedding(torch.from_numpy(x).type(torch.LongTensor), torch.from_numpy(word_embedding))
    x = F.embedding(x.type(torch.LongTensor), word_embedding).to(device)
    return x

############################################ ATTENTION #######################################################
class Attention(nn.Module):
    def __init__(self, n_hidden, sen_len):
        super(Attention, self).__init__()
        self.n_hidden = n_hidden
        self.sen_len = sen_len
        self.linear1 = nn.Linear(n_hidden*2, n_hidden*2)
        self.linear2 = nn.Linear(n_hidden*2, 1)

    def forward(self, x):
        '''
        input shape: [batch_size * doc_len, sen_len, 2 * n_hidden]
        output shape: [batch_size * doc_len, 2 * n_hidden]
        '''
        x_tmp = x.reshape(-1, self.n_hidden*2)
        # x_tmp is of shape (batch_size * doc_len * sen_len, 2 * n_hidden)
        u = torch.tanh(self.linear1(x_tmp))
        # u is of shape (batch_size * doc_len * sen_len, 2 * n_hidden)
        alpha = self.linear2(u)
        # alpha is of shape (batch_size * doc_len * sen_len, 1)
        alpha = F.softmax(alpha.reshape(-1, 1, self.sen_len), dim = -1)
        # alpha is of shape (batch_size * doc_len, 1, sen_len)
        x = torch.matmul(alpha, x).reshape(-1, self.n_hidden*2)
        # x is of shape (batch_size * doc_len, 2 * n_hidden)
        return x

############################################## GET MASK ######################################################
def getmask(y, doc_len):
    '''
    input(s) shape: [max_doc_len * max_doc_len, 2], doc_len
    output shape: [doc_len * doc_len, 2]
    '''
    i = 0; j = 0
    max_doc_len = int(np.sqrt(y.shape[0]))
    y_mask = torch.zeros(doc_len * doc_len, 2)
    while j < doc_len**2:
        y_mask[j : j+doc_len] = y[i : i+doc_len]
        j += doc_len; i += max_doc_len
    return y_mask

############################################ LOSS FUNCTION ###################################################
class ce_loss_aux(torch.nn.Module):
    def __init__(self):
        super(ce_loss_aux, self).__init__()

    def forward(self, y_true, y_pred, doc_len, diminish_factor=1.0):
        '''
        input(s) shape: [batch_size, doc_len, 2]
        output shape: []
        '''
        y_true = y_true.to('cpu'); y_pred = y_pred.to('cpu')
        loss = torch.autograd.Variable(torch.zeros([], dtype=torch.double))
        for i in range(len(doc_len)):
            y_true_masked = y_true[i, :doc_len[i]]; y_pred_masked = y_pred[i, :doc_len[i]].double()
            y_pred_masked_ones = torch.log(y_pred_masked[:, 1][y_true_masked[:, 1]==1.])
            y_pred_masked_zeros = torch.log(y_pred_masked[:, 0][y_true_masked[:, 0]==1.])
            pos_loss = -torch.sum(y_pred_masked_ones)
            neg_loss = -torch.sum(y_pred_masked_zeros)*diminish_factor
            loss += pos_loss + neg_loss
            # loss -= torch.sum(y_true_masked * torch.log(y_pred_masked))
        loss /= torch.sum(doc_len)
        loss.requires_grad_(True)
        return loss

class ce_loss_pair(torch.nn.Module):
    def __init__(self, diminish_factor):
        super(ce_loss_pair, self).__init__()
        self.diminish_factor = diminish_factor

    def forward(self, y_true, y_pred, doc_len):
        '''
        input(s) shape: [batch_size, doc_len, 2]
        output shape: []
        '''
        y_true = y_true.to('cpu'); y_pred = y_pred.to('cpu')
        loss = torch.autograd.Variable(torch.zeros([], dtype=torch.double))
        for i in range(len(doc_len)):
            y_true_masked = getmask(y_true[i].clone(), doc_len[i])
            y_pred_masked = getmask(y_pred[i].clone(), doc_len[i]).double()
            y_pred_masked_ones = torch.log(y_pred_masked[:, 1][y_true_masked[:, 1]==1])
            pos_loss = -torch.sum(y_pred_masked_ones)
            y_pred_masked_zeros = torch.log(y_pred_masked[:, 0][y_true_masked[:, 0]==1])
            neg_loss = -torch.sum(y_pred_masked_zeros)
            ############################## Give less weight to -ve examples ##############################
            neg_loss *= self.diminish_factor
            loss += pos_loss + neg_loss
            # y_pred_masked needs to be of dtype double
        loss /= torch.sum(doc_len)
        loss.requires_grad_(True)
        return loss

########################################## CREATE PAIRS ######################################################
def create_pairs(x1, x2):
    '''
    input(s) shape: [batch_size, doc_len, 2 * n_hidden]
    output shape: [batch_size, doc_len * doc_len, 4 * n_hidden]
    '''
    iters = 0
    for i in range(x1.shape[1]):
        for j in range(x2.shape[1]):
            x3_tmp = torch.cat([x1[:, i, :], x2[:, j, :]], dim=1).unsqueeze(1)
            if iters :
                x3 = torch.cat([x3, x3_tmp], dim=1)
            else :
                x3 = x3_tmp
            iters += 1
    return x3

############################################ DATA GEN ########################################################
def batch_index(length, batch_size, test=False):
    index = list(range(length))
    if test == False:
        np.random.shuffle(index)
    for i in range(int((length + batch_size -1)/batch_size)):
        ret = index[i * batch_size : (i + 1) * batch_size]
        if test == False and len(ret) < batch_size : break
        yield ret

def get_batch_data_pair(x, sen_len, doc_len, y_position, y_cause, y_pair, distance, batch_size, test=False):
    for index in batch_index(len(y_cause), batch_size, test):
        feed_list = [x[index], sen_len[index], doc_len[index], y_position[index], y_cause[index], \
        y_pair[index], distance[index]]
        yield feed_list, len(index)

###################################### ACCURACY, PRECISION, RECALL, F1 #######################################
def metrics(y_true, y_pred):
    y_true_cpu = y_true.cpu().numpy()
    y_pred_cpu = y_pred.cpu().numpy()
    true_pos = np.sum((y_true_cpu == 1.) & (y_pred_cpu == 1.))
    true_neg = np.sum((y_true_cpu == 0.) & (y_pred_cpu == 0.))
    false_pos = np.sum((y_true_cpu == 0.) & (y_pred_cpu == 1.))
    false_neg = np.sum((y_true_cpu == 1.) & (y_pred_cpu == 0.))
    epsilon = 1e-9
    acc = (true_pos + true_neg) / (false_pos + false_neg + true_pos + true_neg + epsilon)
    p = true_pos / (false_pos + true_pos + epsilon)
    r = true_pos / (false_neg + true_pos + epsilon)
    f1 = 2 * p * r / (p + r + epsilon)
    return acc, p, r, f1

def acc_prf_aux(pred_y, true_y, doc_len, average='weighted'):
    _, true_indices = torch.max(true_y, 2)
    _, pred_indices = torch.max(pred_y, 2)
    true_indices_masked = []
    pred_indices_masked = []
    for i in range(len(doc_len)):
        true_indices_masked.extend(true_indices[i, :doc_len[i]])
        pred_indices_masked.extend(pred_indices[i, :doc_len[i]])

    true_indices_masked = torch.tensor(true_indices_masked).to(true_y.device)
    pred_indices_masked = torch.tensor(pred_indices_masked).to(pred_y.device)

    acc, p, r, f1 = metrics(true_indices_masked, pred_indices_masked)
    return acc, p, r, f1

def acc_prf_pair(pred_y, true_y, doc_len):
    true_indices_masked_list = []
    pred_indices_masked_list = []

    for i in range(len(doc_len)):
        true_y_masked = getmask(true_y[i].clone(), doc_len[i])
        pred_y_masked = getmask(pred_y[i].clone(), doc_len[i])

        _, true_indices_masked = torch.max(true_y_masked, 1)
        _, pred_indices_masked = torch.max(pred_y_masked, 1)

        true_indices_masked_list.extend(true_indices_masked.tolist())  # Convert to Python list
        pred_indices_masked_list.extend(pred_indices_masked.tolist())  # Convert to Python list

    # Now you can convert these lists to tensors when needed
    true_indices_masked_tensor = torch.tensor(true_indices_masked_list).to(device)
    pred_indices_masked_tensor = torch.tensor(pred_indices_masked_list).to(device)

    acc, p, r, f1 = metrics(true_indices_masked_tensor, pred_indices_masked_tensor)
    return acc, p, r, f1

utils>prepare_data.py


In [ ]:
############################################ IMPORT ##########################################################
import sys, os
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score
import torch
from torch import nn
from torch import optim
from torch.nn import functional as F

############################################ LOAD W2V EMBEDDING ###############################################
def load_w2v(embedding_dim, embedding_dim_pos, train_file_path, embedding_path):
    print('\nload embedding...')

    words = []
    inputFile1 = open(train_file_path, 'r',encoding='cp1252')
    for line in inputFile1.readlines():
        line = line.strip().split(',')
        emotion, clause = line[2], line[-2]
        words.extend(emotion.lower().split() + clause.lower().split())
        # words extended by ['happy','the','thief','was','caught']

    words = set(words) # Collection of all unique words
    word_idx = dict((c, k + 1) for k, c in enumerate(words)) # Each word and its position
    word_idx_rev = dict((k + 1, c) for k, c in enumerate(words)) # Each word and its position

    w2v = {}
    inputFile2 = open(embedding_path, 'r')
    inputFile2.readline()
    for line in inputFile2.readlines():
        line = line.strip().split(' ')
        w, ebd = line[0], line[1:]
        w2v[w] = ebd

    embedding = [list(np.zeros(embedding_dim))]
    hit = 0
    for item in words:
        if item in w2v:
            vec = list(map(float, w2v[item]))
            hit += 1
        else:
            vec = list(np.random.rand(embedding_dim) / 5. - 0.1)
            # Randomly take from the uniform distribution [-0.1, 0.1]
        embedding.append(vec)
    print('w2v_file: {}\nall_words: {} hit_words: {}'.format(embedding_path, len(words), hit))
    # add a noisy embedding in the end for out of vocabulary words
    embedding.extend([list(np.random.rand(embedding_dim) / 5. - 0.1)])

    embedding_pos = [list(np.zeros(embedding_dim_pos))]
    embedding_pos.extend([list(np.random.normal(loc=0.0, scale=0.1, size=embedding_dim_pos)) \
        for i in range(200)])
    embedding, embedding_pos = np.array(embedding), np.array(embedding_pos)

    print("embedding.shape: {} embedding_pos.shape: {}".format(embedding.shape, embedding_pos.shape))
    print("load embedding done!\n")
    return word_idx_rev, word_idx, embedding, embedding_pos

############################################ LOAD DATA PAIR STEP ##############################################
def load_data_pair(input_file, word_idx, max_doc_len=75, max_sen_len=45):
    print('load data_file: {}'.format(input_file))
    pair_id_all, y_emotion, y_cause, y_pair, x, sen_len, doc_len, distance = [], [], [], [], [], [], [], []
    n_cut = 0
    inputFile = open(input_file, 'r',encoding='cp1252')

    while True:
        line = inputFile.readline()
        if line == '':
            break

        line = line.strip().split()
        doc_id = int(line[0])
        d_len = int(line[1])

        # Skipping lines if the document length is greater than or equal to max_doc_len
        if d_len >= max_doc_len:
            for i in range(d_len + 1):
                line = inputFile.readline().strip().split(',')
            continue

        pairs_str = inputFile.readline().strip()

        # Check if pairs_str is not empty
        if pairs_str:
            pairs = eval(pairs_str)

            # Always convert pairs to a list of lists for consistency
            if isinstance(pairs[0], tuple):
                pos_list, cause_list = zip(*pairs)
                pos_list = list(pos_list)  # Convert pos_list to a list
                cause_list = list(cause_list)  # Convert cause_list to a list
                pairs = list(map(list, pairs))
            else:
                # Handle the case where there is only one emotion-cause pair
                pos_list, cause_list = pairs
                pos_list = [pos_list]  # Convert pos_list to a list
                cause_list = [cause_list]  # Convert cause_list to a list
                pairs = [list(pairs)]



    # Continue with the rest of your code, using the 'pairs' variable as needed
        pair_id_all.extend([doc_id * 10000 + p[0] * 100 + p[1] for p in pairs])

        y_emotion_tmp, y_cause_tmp, y_pair_tmp, sen_len_tmp, x_tmp, distance_tmp = \
            np.zeros((max_doc_len, 2)), np.zeros((max_doc_len, 2)), np.zeros((max_doc_len * max_doc_len, 2)), \
            np.zeros((max_doc_len,)), np.zeros((max_doc_len, max_sen_len)), np.zeros((max_doc_len * max_doc_len,))

        for i in range(d_len):
            line = inputFile.readline().strip().split(',')
            words = line[-1]
            sen_len_tmp[i] = min(len(words.split()), max_sen_len)

            for j, word in enumerate(words.split()):
                word = word.lower()
                if j >= max_sen_len:
                    n_cut += 1
                    break
                elif word not in word_idx:
                    x_tmp[i][j] = 0  # Change this to 0 for 'unknown' in the word_idx
                else:
                    x_tmp[i][j] = int(word_idx[word])

        for i in range(d_len):
            for j in range(d_len):
                # Check whether i is an emotion clause
                if i + 1 in pos_list:
                    y_emotion_tmp[i][0] = 0
                    y_emotion_tmp[i][1] = 1
                else:
                    y_emotion_tmp[i][0] = 1
                    y_emotion_tmp[i][1] = 0

                # Check whether j is a cause clause
                if j + 1 in cause_list:
                    y_cause_tmp[j][0] = 0
                    y_cause_tmp[j][1] = 1
                else:
                    y_cause_tmp[j][0] = 1
                    y_cause_tmp[j][1] = 0

                # Check whether i, j clauses are emotion cause pairs
                pair_id_curr = doc_id * 10000 + (i + 1) * 100 + (j + 1)
                if pair_id_curr in pair_id_all:
                    y_pair_tmp[i * max_doc_len + j][0] = 0
                    y_pair_tmp[i * max_doc_len + j][1] = 1
                else:
                    y_pair_tmp[i * max_doc_len + j][0] = 1
                    y_pair_tmp[i * max_doc_len + j][1] = 0

                # Find the distance between the clauses, and use the same embedding beyond 10 clauses
                distance_tmp[i * max_doc_len + j] = min(max(j - i + 100, 90), 110)

        y_emotion.append(y_emotion_tmp)
        y_cause.append(y_cause_tmp)
        y_pair.append(y_pair_tmp)
        x.append(x_tmp)
        sen_len.append(sen_len_tmp)
        doc_len.append(d_len)
        distance.append(distance_tmp)

    y_emotion, y_cause, y_pair, x, sen_len, doc_len, distance = map(torch.tensor, \
        [y_emotion, y_cause, y_pair, x, sen_len, doc_len, distance])

    for var in ['y_emotion', 'y_cause', 'y_pair', 'x', 'sen_len', 'doc_len', 'distance']:
        print('{}.shape {}'.format(var, eval(var).shape))
    print('n_cut {}'.format(n_cut))
    print('load data done!\n')
    return y_emotion, y_cause, y_pair, x, sen_len, doc_len, distance

In [ ]:
# train_file_path = 'clause_keywords.csv'          # clause keyword file
# w2v_file = 'w2v_200.txt'                         # embedding file
# max_sen_len = 30                                                    # max number of tokens per sentence
# max_doc_len = 41
# embedding_dim = 200                                                 # dimension of word embedding
# embedding_dim_pos = 50

# word_idx_rev, word_id_mapping, word_embedding, pos_embedding = load_w2v(
#         embedding_dim, embedding_dim_pos, train_file_path, w2v_file)
# word_embedding = torch.from_numpy(word_embedding)
# train_file_name = 'fold1_train.txt'
# ans = load_data_pair(train_file_name, word_id_mapping, max_doc_len, max_sen_len)
# print(ans)

E2E_PextE.py


In [ ]:
def save_failed_examples(failed_examples, filename):
    with open(filename, 'w') as f:
        json.dump(failed_examples, f, indent=4)

In [ ]:
############################################ IMPORT ##########################################################
import sys, os
import numpy as np
import torch
from torch import nn
from torch import optim
from torch.nn import functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

############################################ FLAGS ############################################################
train_file_path = './data_combine_eng/clause_keywords.csv'          # clause keyword file
w2v_file = './data_combine_eng/ECF_glove_300.txt'                         # embedding file
embedding_dim = 300                                                 # dimension of word embedding
embedding_dim_pos = 50                                              # dimension of position embedding
max_sen_len = 30                                                    # max number of tokens per sentence
max_doc_len = 41                                                    # max number of tokens per document
n_hidden = 100                                                      # number of hidden unit
n_class = 2                                                         # number of distinct class
training_epochs = 15                                                # number of train epochs
batch_size = 32                                                     # number of example per batch
learning_rate = 0.0050                                              # learning rate
keep_prob1 = 0.8                                                    # word embedding training dropout keep prob
keep_prob2 = 1.0                                                    # softmax layer dropout keep prob
keep_prob3 = 1.0                                                    # softmax layer dropout keep prob
l2_reg = 0.00010                                                    # l2 regularization
cause = 1.0                                                         # lambda1
pos = 1.0                                                           # lambda2
pair = 2.5                                                          # lambda3
diminish_factor = 0.400                                             # give less weight to -ve examples

############################################ MODEL ############################################################
class E2E_PextE(nn.Module):
    def __init__(self, embedding_dim, embedding_dim_pos, sen_len, doc_len, keep_prob1, keep_prob2, \
                 keep_prob3, n_hidden, n_class):
        super(E2E_PextE, self).__init__()
        self.embedding_dim = embedding_dim; self.embedding_dim_pos = embedding_dim_pos
        self.sen_len = sen_len; self.doc_len = doc_len
        self.keep_prob1 = keep_prob1; self.keep_prob2 = keep_prob2
        self.n_hidden = n_hidden; self.n_class = n_class

        self.dropout1 = nn.Dropout(p = 1 - keep_prob1)
        self.dropout2 = nn.Dropout(p = 1 - keep_prob2)
        self.dropout3 = nn.Dropout(p = 1 - keep_prob3)
        self.relu = nn.ReLU()
        self.pos_linear = nn.Linear(2*n_hidden, n_class)
        self.cause_linear = nn.Linear(2*n_hidden, n_class)
        self.pair_linear1 = nn.Linear(4*n_hidden + embedding_dim_pos, n_hidden//2)
        self.pair_linear2 = nn.Linear(n_hidden//2, n_class)
        self.word_bilstm = nn.LSTM(embedding_dim, n_hidden, batch_first = True, bidirectional = True)
        self.cause_bilstm = nn.LSTM(2*n_hidden + n_class, n_hidden, batch_first = True, bidirectional = True)
        self.pos_bilstm = nn.LSTM(2*n_hidden, n_hidden, batch_first = True, bidirectional = True)
        self.attention = Attention(n_hidden, sen_len)

    def get_clause_embedding(self, x):
        '''
        input shape: [batch_size, doc_len, sen_len, embedding_dim]
        output shape: [batch_size, doc_len, 2 * n_hidden]
        '''
        x = x.reshape(-1, self.sen_len, self.embedding_dim)
        x = self.dropout1(x)
        # x is of shape (batch_size * max_doc_len, max_sen_len, embedding_dim)
        x, hidden_states = self.word_bilstm(x.float())
        # x is of shape (batch_size * max_doc_len, max_sen_len, 2 * n_hidden)
        s = self.attention(x).reshape(-1, self.doc_len, 2 * self.n_hidden)
        # s is of shape (batch_size, max_doc_len, 2 * n_hidden)
        return s

    def get_emotion_prediction(self, x):
        '''
        input shape: [batch_size, doc_len, 2 * n_hidden]
        output(s) shape: [batch_size, doc_len, 2 * n_hidden], [batch_size, doc_len, n_class]
        '''
        x_context, hidden_states = self.pos_bilstm(x.float())
        # x_context is of shape (batch_size, max_doc_len, 2 * n_hidden)
        x = x_context.reshape(-1, 2 * self.n_hidden)
        x = self.dropout2(x)
        # x is of shape (batch_size * max_doc_len, 2 * n_hidden)
        pred_pos = F.softmax(self.pos_linear(x), dim = -1)
        # pred_pos is of shape (batch_size * max_doc_len, n_class)
        pred_pos = pred_pos.reshape(-1, self.doc_len, self.n_class)
        # pred_pos is of shape (batch_size * max_doc_len, n_class)
        return x_context, pred_pos

    def get_cause_prediction(self, x):
        '''
        input shape: [batch_size, doc_len, 2 * n_hidden + n_class]
        output(s) shape: [batch_size, doc_len, 2 * n_hidden], [batch_size, doc_len, n_class]
        '''
        x_context, hidden_states = self.cause_bilstm(x.float())
        # x_context is of shape (batch_size, max_doc_len, 2 * n_hidden)
        x = x_context.reshape(-1, 2 * self.n_hidden)
        x = self.dropout2(x)
        # x is of shape (batch_size * max_doc_len, 2 * n_hidden)
        pred_cause = F.softmax(self.cause_linear(x), dim = -1)
        # pred_pos is of shape (batch_size * max_doc_len, n_class)
        pred_cause = pred_cause.reshape(-1, self.doc_len, self.n_class)
        # pred_pos is of shape (batch_size * max_doc_len, n_class)
        return x_context, pred_cause

    def get_pair_prediction(self, x1, x2, distance):
        '''
        input(s) shape: [batch_size * doc_len, 2 * n_hidden], [batch_size * doc_len, 2 * n_hidden],
                        [batch_size, doc_len * doc_len, embedding_dim_pos]
        output shape: [batch_size, doc_len * doc_len, n_class]
        '''
        x = create_pairs(x1, x2)
        # x is of shape (batch_size, max_doc_len * max_doc_len, 4 * n_hidden)
        x_distance = torch.cat([x, distance.float()], -1)
        # x_distance is of shape (batch_size, max_doc_len * max_doc_len, 4 * n_hidden + embedding_dim_pos)
        x_distance = x_distance.reshape(-1, 4 * self.n_hidden + self.embedding_dim_pos)
        x_distance = self.dropout3(x_distance)
        # x is of shape (batch_size * max_doc_len * max_doc_len, 4 * n_hidden + embedding_dim_pos)
        pred_pair = F.softmax(self.pair_linear2(self.relu(self.pair_linear1(x_distance))), dim = -1)
        # pred_pair is of shape (batch_size * max_doc_len * max_doc_len, n_class)
        pred_pair = pred_pair.reshape(-1, self.doc_len * self.doc_len, self.n_class)
        # pred_pair is of shape (batch_size, max_doc_len * max_doc_len, n_class)
        return pred_pair

    def forward(self, x, distance):
        '''
        input(s) shape: [batch_size, doc_len, sen_len, embedding_dim],
                        [batch_size, doc_len * doc_len, embedding_dim_pos]
        output(s) shape: [batch_size, doc_len, n_class], [batch_size, doc_len, n_class],
                         [batch_size, doc_len * doc_len, n_class]
        '''
        s = self.get_clause_embedding(x)
        x_pos, pred_pos = self.get_emotion_prediction(s)
        s_pred_pos = torch.cat([s, pred_pos], 2)
        x_cause, pred_cause = self.get_cause_prediction(s_pred_pos)
        pred_pair = self.get_pair_prediction(x_pos, x_cause, distance)
        return pred_pos, pred_cause, pred_pair

############################################ TRAIN #####################################################
def train_and_eval(Model, pos_cause_criterion, pair_criterion, optimizer):
    word_idx_rev, word_id_mapping, word_embedding, pos_embedding = load_w2v(
        embedding_dim, embedding_dim_pos, train_file_path, w2v_file)
    word_embedding = torch.from_numpy(word_embedding)
    # Train distance embeddings
    pos_embedding = torch.autograd.Variable(torch.from_numpy(pos_embedding))
    pos_embedding.requires_grad_(True)
    torch.save(word_embedding, './save/word_embedding.pth')
    torch.save(word_id_mapping, './save/word_id_mapping.pth')
    acc_cause_list, p_cause_list, r_cause_list, f1_cause_list = [], [], [], []
    acc_pos_list, p_pos_list, r_pos_list, f1_pos_list = [], [], [], []
    acc_pair_list, p_pair_list, r_pair_list, f1_pair_list = [], [], [], []
    #################################### LOOP OVER FOLDS ####################################
    for fold in range(1, 2):
        print('############# fold {} begin ###############'.format(fold))
        ############################# RE-INITIALIZE MODEL PARAMETERS #############################
        for layer in Model.parameters():
            nn.init.uniform_(layer.data, -0.10, 0.10)
        #################################### TRAIN/TEST DATA ####################################
        train_file_name = 'train.txt'.format(fold)
        val_file_name = 'dev.txt'.format(fold)
        tr_y_position, tr_y_cause, tr_y_pair, tr_x, tr_sen_len, tr_doc_len, tr_distance = load_data_pair(
                        './data_combine_eng/'+train_file_name, word_id_mapping, max_doc_len, max_sen_len)
        val_y_position, val_y_cause, val_y_pair, val_x, val_sen_len, val_doc_len, val_distance = \
            load_data_pair('./data_combine_eng/'+val_file_name, word_id_mapping, max_doc_len, max_sen_len)
        max_f1_cause, max_f1_pos, max_f1_pair, max_f1_avg = [-1.] * 4
        #################################### LOOP OVER EPOCHS ####################################
        for epoch in range(1, training_epochs + 1):
            step = 1
            #################################### GET BATCH DATA ####################################
            for train, _ in get_batch_data_pair(
                tr_x, tr_sen_len, tr_doc_len, tr_y_position, tr_y_cause, tr_y_pair, tr_distance, batch_size):
                tr_x_batch, tr_sen_len_batch, tr_doc_len_batch, tr_true_y_pos, tr_true_y_cause, \
                tr_true_y_pair, tr_distance_batch = train
                Model.train()
                tr_pred_y_pos, tr_pred_y_cause, tr_pred_y_pair = Model(embedding_lookup(word_embedding, \
                tr_x_batch), embedding_lookup(pos_embedding, tr_distance_batch))
                ############################## LOSS FUNCTION AND OPTIMIZATION ##############################
                loss = pos_cause_criterion(tr_true_y_pos, tr_pred_y_pos, tr_doc_len_batch)*pos + \
                pos_cause_criterion(tr_true_y_cause, tr_pred_y_cause, tr_doc_len_batch)*cause + \
                pair_criterion(tr_true_y_pair, tr_pred_y_pair, tr_doc_len_batch)*pair
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                #################################### PRINT AFTER EPOCHS ####################################
                if step % 25 == 0:
                    # print(Model.pair_linear.weight.shape); print(Model.pair_linear.weight.grad)
                    print('Fold {}, Epoch {}, step {}: train loss {:.4f} '.format(fold, epoch, step, loss))
                    acc, p, r, f1 = acc_prf_aux(tr_pred_y_pos, tr_true_y_pos, tr_doc_len_batch)
                    print('emotion_predict: train acc {:.4f} p {:.4f} r {:.4f} f1 score {:.4f}'.format(
                            acc, p, r, f1))
                    acc, p, r, f1 = acc_prf_aux(tr_pred_y_cause, tr_true_y_cause, tr_doc_len_batch)
                    print('cause_predict: train acc {:.4f} p {:.4f} r {:.4f} f1 score {:.4f}'.format(
                            acc, p, r, f1))
                    acc, p, r, f1 = acc_prf_pair(tr_pred_y_pair, tr_true_y_pair, tr_doc_len_batch)
                    print('pair_predict: train acc {:.4f} p {:.4f} r {:.4f} f1 score {:.4f}'.format(
                            acc, p, r, f1))
                step += 1
            #################################### TEST ON 1 FOLD ####################################
            with torch.no_grad():
                Model.eval()
                val_pred_y_pos, val_pred_y_cause, val_pred_y_pair = Model(embedding_lookup(word_embedding, \
                val_x), embedding_lookup(pos_embedding, val_distance))

                loss = pos_cause_criterion(val_y_position, val_pred_y_pos, val_doc_len)*pos + \
                pos_cause_criterion(val_y_cause, val_pred_y_cause, val_doc_len)*cause + \
                pair_criterion(val_y_pair, val_pred_y_pair, val_doc_len)*pair
                print('Fold {} val loss {:.4f}'.format(fold, loss))
                acc, p, r, f1 = acc_prf_aux(val_pred_y_pos, val_y_position, val_doc_len)
                result_avg_pos = [acc, p, r, f1]
                if f1 > max_f1_pos:
                    max_acc_pos, max_p_pos, max_r_pos, max_f1_pos = acc, p, r, f1
                print('emotion_predict: val acc {:.4f} p {:.4f} r {:.4f} f1 {:.4f}'.format(acc, p, r, f1))
                print('max_acc {:.4f} max_p {:.4f} max_r {:.4f} max_f1 {:.4f}\n'.format(
                    max_acc_pos, max_p_pos, max_r_pos, max_f1_pos))

                acc, p, r, f1 = acc_prf_aux(val_pred_y_cause, val_y_cause, val_doc_len)
                result_avg_cause = [acc, p, r, f1]
                if f1 > max_f1_cause:
                    max_acc_cause, max_p_cause, max_r_cause, max_f1_cause = acc, p, r, f1
                print('cause_predict: val acc {:.4f} p {:.4f} r {:.4f} f1 {:.4f}'.format(acc, p, r, f1))
                print('max_acc {:.4f} max_p {:.4f} max_r {:.4f} max_f1 {:.4f}\n'.format(
                    max_acc_cause, max_p_cause, max_r_cause, max_f1_cause))

                acc, p, r, f1 = acc_prf_pair(val_pred_y_pair, val_y_pair, val_doc_len)
                result_avg_pair = [acc, p, r, f1]
                if f1 > max_f1_pair:
                    max_acc_pair, max_p_pair, max_r_pair, max_f1_pair = acc, p, r, f1
                print('pair_predict: val acc {:.4f} p {:.4f} r {:.4f} f1 {:.4f}'.format(acc, p, r, f1))
                print('max_acc {:.4f} max_p {:.4f} max_r {:.4f} max_f1 {:.4f}\n'.format(
                    max_acc_pair, max_p_pair, max_r_pair, max_f1_pair))

            #################################### STORE BETTER PAIR F1 ####################################
            if result_avg_pair[-1] > max_f1_avg:
                torch.save(pos_embedding, "./save/pos_embedding.pth".format(fold))
                torch.save(Model.state_dict(), "./save/E2E-PextE.pth".format(fold))
                max_f1_avg = result_avg_pair[-1]
                result_avg_cause_max = result_avg_cause
                result_avg_pos_max = result_avg_pos
                result_avg_pair_max = result_avg_pair

            print('avg max cause: max_acc {:.4f} max_p {:.4f} max_r {:.4f} max_f1 {:.4f}'.format(
                result_avg_cause_max[0], result_avg_cause_max[1], result_avg_cause_max[2], result_avg_cause_max[3]))
            print('avg max pos: max_acc {:.4f} max_p {:.4f} max_r {:.4f} max_f1 {:.4f}'.format(
                result_avg_pos_max[0], result_avg_pos_max[1], result_avg_pos_max[2], result_avg_pos_max[3]))
            print('avg max pair: max_acc {:.4f} max_p {:.4f} max_r {:.4f} max_f1 {:.4f}\n'.format(
                result_avg_pair_max[0], result_avg_pair_max[1], result_avg_pair_max[2], result_avg_pair_max[3]))

        print('############# fold {} end ###############'.format(fold))
        acc_cause_list.append(result_avg_cause_max[0])
        p_cause_list.append(result_avg_cause_max[1])
        r_cause_list.append(result_avg_cause_max[2])
        f1_cause_list.append(result_avg_cause_max[3])
        acc_pos_list.append(result_avg_pos_max[0])
        p_pos_list.append(result_avg_pos_max[1])
        r_pos_list.append(result_avg_pos_max[2])
        f1_pos_list.append(result_avg_pos_max[3])
        acc_pair_list.append(result_avg_pair_max[0])
        p_pair_list.append(result_avg_pair_max[1])
        r_pair_list.append(result_avg_pair_max[2])
        f1_pair_list.append(result_avg_pair_max[3])

    #################################### FINAL TEST RESULTS ON 10 FOLDS ####################################
    all_results = [acc_cause_list, p_cause_list, r_cause_list, f1_cause_list, \
    acc_pos_list, p_pos_list, r_pos_list, f1_pos_list, acc_pair_list, p_pair_list, r_pair_list, f1_pair_list,]
    acc_cause, p_cause, r_cause, f1_cause, acc_pos, p_pos, r_pos, f1_pos, acc_pair, p_pair, r_pair, f1_pair = \
        map(lambda x: np.array(x).mean(), all_results)
    print('\ncause_predict: val f1 in 10 fold: {}'.format(np.array(f1_cause_list).reshape(-1,1)))
    print('average : acc {:.4f} p {:.4f} r {:.4f} f1 {:.4f}\n'.format(acc_cause, p_cause, r_cause, f1_cause))
    print('emotion_predict: val f1 in 10 fold: {}'.format(np.array(f1_pos_list).reshape(-1,1)))
    print('average : acc {:.4f} p {:.4f} r {:.4f} f1 {:.4f}\n'.format(acc_pos, p_pos, r_pos, f1_pos))
    print('pair_predict: val f1 in 10 fold: {}'.format(np.array(f1_pair_list).reshape(-1,1)))
    print('average : acc {:.4f} p {:.4f} r {:.4f} f1 {:.4f}\n'.format(acc_pair, p_pair, r_pair, f1_pair))

############################################### MAIN ########################################################
def main():
    Model = E2E_PextE(embedding_dim, embedding_dim_pos, max_sen_len, max_doc_len, \
    keep_prob1, keep_prob2, keep_prob3, n_hidden, n_class)
    Model.to(device)
    print(Model)
    x = torch.rand([batch_size, max_doc_len, max_sen_len, embedding_dim]).to(device)
    distance = torch.rand([batch_size, max_doc_len * max_doc_len, embedding_dim_pos]).to(device)
    pred_pos, pred_cause, pred_pair = Model(x, distance)
    print("Random i/o shapes x: {}, distance: {}, y_pos: {}, y_cause: {}, y_pair: {}".format(
        x.shape, distance.shape, pred_pos.shape, pred_cause.shape, pred_pair.shape))
    pos_cause_criterion = ce_loss_aux(); pair_criterion = ce_loss_pair(diminish_factor)
    optimizer = optim.Adam(Model.parameters(), lr=learning_rate, weight_decay=l2_reg)
    train_and_eval(Model, pos_cause_criterion, pair_criterion, optimizer)

if __name__ == "__main__":
    main()

E2E_PextE(
  (dropout1): Dropout(p=0.19999999999999996, inplace=False)
  (dropout2): Dropout(p=0.0, inplace=False)
  (dropout3): Dropout(p=0.0, inplace=False)
  (relu): ReLU()
  (pos_linear): Linear(in_features=200, out_features=2, bias=True)
  (cause_linear): Linear(in_features=200, out_features=2, bias=True)
  (pair_linear1): Linear(in_features=450, out_features=50, bias=True)
  (pair_linear2): Linear(in_features=50, out_features=2, bias=True)
  (word_bilstm): LSTM(300, 100, batch_first=True, bidirectional=True)
  (cause_bilstm): LSTM(202, 100, batch_first=True, bidirectional=True)
  (pos_bilstm): LSTM(200, 100, batch_first=True, bidirectional=True)
  (attention): Attention(
    (linear1): Linear(in_features=200, out_features=200, bias=True)
    (linear2): Linear(in_features=200, out_features=1, bias=True)
  )
)
Random i/o shapes x: torch.Size([32, 41, 30, 300]), distance: torch.Size([32, 1681, 50]), y_pos: torch.Size([32, 41, 2]), y_cause: torch.Size([32, 41, 2]), y_pair: torch.Size

Test code

In [ ]:

############################################ IMPORT ##########################################################
import sys, os
import numpy as np
import torch
from torch import nn
from torch import optim
from torch.nn import functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

############################################ FLAGS ############################################################
train_file_path = './data_combine_eng/clause_keywords.csv'          # clause keyword file
w2v_file = 'data_combine_eng/ECF_glove_300.txt'                         # embedding file
embedding_dim = 300                                                 # dimension of word embedding
embedding_dim_pos = 50                                              # dimension of position embedding
max_sen_len = 30                                                    # max number of tokens per sentence
max_doc_len = 41                                                    # max number of tokens per document
n_hidden = 100                                                      # number of hidden unit
n_class = 2                                                         # number of distinct class
keep_prob1 = 0.8                                                    # word embedding training dropout keep prob
keep_prob2 = 1.0                                                    # softmax layer dropout keep prob
keep_prob3 = 1.0                                                    # softmax layer dropout keep prob

################################################ TEST #########################################################
def test(Model):
    word_embedding = torch.load("./save/word_embedding.pth")
    word_id_mapping = torch.load("./save/word_id_mapping.pth")
    acc_cause_list, p_cause_list, r_cause_list, f1_cause_list = [], [], [], []
    acc_pos_list, p_pos_list, r_pos_list, f1_pos_list = [], [], [], []
    acc_pair_list, p_pair_list, r_pair_list, f1_pair_list = [], [], [], []
    #################################### LOOP OVER FOLDS ####################################
    for fold in range(1, 2):
        print('############# fold {} begin ###############'.format(fold))
        #################################### LOAD TEST DATA ####################################
        test_file_name = 'test.txt'.format(fold)
        te_y_position, te_y_cause, te_y_pair, te_x, te_sen_len, te_doc_len, te_distance = load_data_pair(
            './data_combine_eng/'+test_file_name, word_id_mapping, max_doc_len, max_sen_len)
        pos_embedding = torch.load("./save/pos_embedding.pth".format(fold))
        Model.load_state_dict(torch.load("./save/E2E-PextE.pth".format(fold)))
        with torch.no_grad():
            Model.eval()
            te_pred_y_pos, te_pred_y_cause, te_pred_y_pair = Model(embedding_lookup(word_embedding, \
                te_x), embedding_lookup(pos_embedding, te_distance))
            # emotion results
            acc, p, r, f1 = acc_prf_aux(te_pred_y_pos, te_y_position, te_doc_len)
            acc_pos_list.append(acc); p_pos_list.append(p); r_pos_list.append(r); f1_pos_list.append(f1)
            print("Fold {} emotion acc: {:.4f} p: {:.4f} r: {:.4f} f1: {:.4f}".format(fold, acc, p, r, f1))
            # cause results
            acc, p, r, f1 = acc_prf_aux(te_pred_y_cause, te_y_cause, te_doc_len)
            acc_cause_list.append(acc); p_cause_list.append(p); r_cause_list.append(r); f1_cause_list.append(f1)
            print("Fold {} cause acc: {:.4f} p: {:.4f} r: {:.4f} f1: {:.4f}".format(fold, acc, p, r, f1))
            # pair results
            acc, p, r, f1 = acc_prf_pair(te_pred_y_pair, te_y_pair, te_doc_len)
            acc_pair_list.append(acc); p_pair_list.append(p); r_pair_list.append(r); f1_pair_list.append(f1)
            print("Fold {} pair acc: {:.4f} p: {:.4f} r: {:.4f} f1:{:.4f}".format(fold, acc, p, r, f1))

        print('############# fold {} end ###############\n'.format(fold))

    #################################### FINAL TEST RESULTS ON 10 FOLDS ####################################
    all_results = [acc_cause_list, p_cause_list, r_cause_list, f1_cause_list, \
    acc_pos_list, p_pos_list, r_pos_list, f1_pos_list, acc_pair_list, p_pair_list, r_pair_list, f1_pair_list,]
    acc_cause, p_cause, r_cause, f1_cause, acc_pos, p_pos, r_pos, f1_pos, acc_pair, p_pair, r_pair, f1_pair = \
        map(lambda x: np.array(x).mean(), all_results)
    print('\ncause_predict: test f1 in 10 fold: {}'.format(np.array(f1_cause_list).reshape(-1,1)))
    print('\naverage : acc {:.4f} p {:.4f} r {:.4f} f1 {:.4f}'.format(acc_cause, p_cause, r_cause, f1_cause))
    print('\nemotion_predict: test f1 in 10 fold: {}'.format(np.array(f1_pos_list).reshape(-1,1)))
    print('\naverage : acc {:.4f} p {:.4f} r {:.4f} f1 {:.4f}'.format(acc_pos, p_pos, r_pos, f1_pos))
    print('\npair_predict: test f1 in 10 fold: {}'.format(np.array(f1_pair_list).reshape(-1,1)))
    print('\naverage : acc {:.4f} p {:.4f} r {:.4f} f1 {:.4f}'.format(acc_pair, p_pair, r_pair, f1_pair))

############################################### MAIN ########################################################
def main():
    Model = E2E_PextE(embedding_dim, embedding_dim_pos, max_sen_len, max_doc_len, \
    keep_prob1, keep_prob2, keep_prob3, n_hidden, n_class).to(device)
    test(Model)

if __name__ == "__main__":
    main()



############# fold 1 begin ###############
load data_file: ./data_combine_eng/test.txt
y_emotion.shape torch.Size([261, 41, 2])
y_cause.shape torch.Size([261, 41, 2])
y_pair.shape torch.Size([261, 1681, 2])
x.shape torch.Size([261, 41, 30])
sen_len.shape torch.Size([261, 41])
doc_len.shape torch.Size([261])
distance.shape torch.Size([261, 1681])
n_cut 0
load data done!

Fold 1 emotion acc: 0.5320 p: 0.5320 r: 1.0000 f1: 0.6945
Fold 1 cause acc: 0.5136 p: 0.5029 r: 0.5581 f1: 0.5291
Fold 1 pair acc: 0.9248 p: 0.3804 r: 0.5137 f1:0.4371
############# fold 1 end ###############


cause_predict: test f1 in 10 fold: [[0.5290566]]

average : acc 0.5136 p 0.5029 r 0.5581 f1 0.5291

emotion_predict: test f1 in 10 fold: [[0.69447978]]

average : acc 0.5320 p 0.5320 r 1.0000 f1 0.6945

pair_predict: test f1 in 10 fold: [[0.43708016]]

average : acc 0.9248 p 0.3804 r 0.5137 f1 0.4371
